# Демонстрация использования класса `OscillogramToCsvConverter`

Этот ноутбук показывает, как использовать класс `OscillogramToCsvConverter` из `raw_to_csv.raw_to_csv` для преобразования файлов COMTRADE в формат CSV. Этот класс является основным компонентом для подготовки данных для последующего использования, например, в машинном обучении.

Основные возможности, которые будут продемонстрированы:
- Базовое преобразование в CSV.
- Преобразование с вырезанием интересующих областей (вокруг событий, отмеченных ML-сигналами).
- Создание "упрощенного" CSV с агрегированными метками событий.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json
import sys
import csv # For writing sample norm_coef.csv

# Настройка путей для импорта модулей проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from core.oscillogram import Oscillogram 
from normalization.normalization import OscillogramNormalizer
from raw_to_csv.raw_to_csv import OscillogramToCsvConverter

# --- Создание временных директорий ---
base_sample_dir_conv = "temp_sample_data_converter"
source_dir_conv = os.path.join(base_sample_dir_conv, "source_oscillograms_conv")
output_dir_conv_csv = os.path.join(base_sample_dir_conv, "output_csvs")
config_files_dir = os.path.join(base_sample_dir_conv, "config_files")

if os.path.exists(base_sample_dir_conv):
    shutil.rmtree(base_sample_dir_conv)
os.makedirs(source_dir_conv)
os.makedirs(output_dir_conv_csv)
os.makedirs(config_files_dir)

# --- Вспомогательная функция для создания CFG/DAT ---
def create_converter_sample_comtrade(path, name, analog_ch_details, digital_ch_details, freq=50.0, samp_rate=1000.0, dat_rows=100):
    cfg_content = f"SampleStation,ConverterDevice,1999\n"
    cfg_content += f"{len(analog_ch_details) + len(digital_ch_details)},{len(analog_ch_details)}A,{len(digital_ch_details)}D\n"
    ch_idx = 1
    for ch_name, _ in analog_ch_details:
        cfg_content += f"{ch_idx},{ch_name},A,,V,1.0,0.0,0,-32767,32767,1,1,P\n" 
        ch_idx += 1
    for ch_name, _ in digital_ch_details:
        cfg_content += f"{ch_idx},{ch_name},,,1\n"
        ch_idx += 1
    cfg_content += f"{freq}\n1\n{samp_rate},{dat_rows}\n"
    cfg_content += "01/01/2023,00:00:00.000000\n01/01/2023,00:00:00.000000\nASCII\n1.0\n"
    with open(os.path.join(path, f"{name}.cfg"), "w", encoding="utf-8") as f: f.write(cfg_content)

    dat_content = ""
    for i in range(dat_rows):
        dat_content += f"{i+1},{int(i*(1000000/samp_rate))}"
        for _, data_series in analog_ch_details: dat_content += f",{data_series[i % len(data_series)]:.6f}"
        for _, data_series in digital_ch_details: dat_content += f",{data_series[i % len(data_series)]}"
        dat_content += "\n"
    with open(os.path.join(path, f"{name}.dat"), "w", encoding="utf-8") as f: f.write(dat_content)

# --- Создание файлов осциллограмм ---
time_pts = np.linspace(0, 2*np.pi*5, 100)
create_converter_sample_comtrade(source_dir_conv, "osc_conv1",
    analog_ch_details=[("U | Bus-1 | phase: A", np.sin(time_pts)*220)],
    digital_ch_details=[("RawDigital1", np.zeros(100, dtype=int)), ("RawDigital2", np.ones(100, dtype=int))]
)
event_signal = np.zeros(100, dtype=int)
event_signal[40:60] = 1
create_converter_sample_comtrade(source_dir_conv, "osc_conv2_event",
    analog_ch_details=[("I | Bus-1 | phase: B", np.cos(time_pts)*5)],
    digital_ch_details=[("MLsignal_1_1_1_1", event_signal)] 
)

# --- Создание sample_norm_coef.csv ---
norm_data_conv = [
    ["name", "norm", "1Ip_base", "1Ub_base"],
    ["osc_conv1", "YES", 5.0, (220.0*np.sqrt(2)/np.sqrt(3))], 
    ["osc_conv2_event", "YES", 5.0, (220.0*np.sqrt(2)/np.sqrt(3))]
]
sample_norm_coef_path_conv = os.path.join(config_files_dir, "sample_norm_coef_conv.csv")
with open(sample_norm_coef_path_conv, "w", newline="", encoding="utf-8") as f: csv.writer(f).writerows(norm_data_conv)

# --- Создание dict_analog_names.json и dict_discrete_names.json ---
analog_names_path = os.path.join(config_files_dir, "dict_analog_names_conv.json")
analog_content_conv = { "bus1": { "U_A": ["U | Bus-1 | phase: A"], "I_B": ["I | Bus-1 | phase: B"] } }
with open(analog_names_path, "w", encoding="utf-8") as f: json.dump(analog_content_conv, f)

discrete_names_path = os.path.join(config_files_dir, "dict_discrete_names_conv.json")
discrete_content_conv = { "bus1": { "ML_EVENT_RAW_1111": ["MLsignal_1_1_1_1"] } }
with open(discrete_names_path, "w", encoding="utf-8") as f: json.dump(discrete_content_conv, f)

print(f"Созданы директории и файлы для демонстрации `OscillogramToCsvConverter` в: {base_sample_dir_conv}")

## 1. Инициализация компонентов
Для работы `OscillogramToCsvConverter` требуется экземпляр `OscillogramNormalizer` (для нормализации данных) и пути к конфигурационным JSON-файлам, определяющим, как сырые имена каналов COMTRADE отображаются на стандартные имена в шинах (`dict_analog_names.json`, `dict_discrete_names.json`).

In [ ]:
# 1. Normalizer
normalizer_conv = OscillogramNormalizer(
    norm_coef_file_path=sample_norm_coef_path_conv, 
    is_print_message=True
)

# 2. OscillogramToCsvConverter
converter = OscillogramToCsvConverter(
    normalizer=normalizer_conv,
    raw_path=source_dir_conv,
    csv_path=output_dir_conv_csv,
    uses_buses=['1'], 
    dict_analog_names_path=analog_names_path,
    dict_discrete_names_path=discrete_names_path,
    is_print_message=True
)
converter.number_periods = 2 
print("\nOscillogramToCsvConverter и зависимости инициализированы.")

## 2. Базовое преобразование в CSV (`create_csv`)

In [ ]:
output_csv_basic = "basic_converted_data.csv"
print(f"\nЗапуск базового преобразования в CSV (выход: {output_csv_basic})...")
df_basic = converter.create_csv(csv_name=output_csv_basic, is_cut_out_area=False, is_simple_csv=False)

if df_basic is not None and not df_basic.empty:
    print(f"\nПервые строки базового CSV ({output_csv_basic}):")
    try: from IPython.display import display; display(df_basic.head()) 
    except ImportError: print(df_basic.head())
    print(f"\nКолонки: {df_basic.columns.tolist()}")
else:
    print(f"Базовый CSV не был создан или пуст.")

## 3. Преобразование с вырезанием области события (`create_csv(is_cut_out_area=True)`)
Для этого в `osc_conv2_event.cfg` был добавлен сигнал `MLsignal_1_1_1_1`, который `OscillogramToCsvConverter` (через `rename_bus_columns` и `get_ml_signals`) должен распознать как событие "Оперативное включение, запуск двигателя" (`ML_1_1_1`). Метод `cut_out_area` оставит данные вокруг этого события.

In [ ]:
output_csv_cut = "cut_area_data.csv"
print(f"\nЗапуск преобразования с вырезанием области (выход: {output_csv_cut})...")
df_cut = converter.create_csv(csv_name=output_csv_cut, is_cut_out_area=True, is_simple_csv=False)

if df_cut is not None and not df_cut.empty:
    print(f"\nДанные из CSV с вырезанной областью ({output_csv_cut}):")
    print("Уникальные имена файлов в CSV (должны содержать '_event N'):")
    print(df_cut['file_name'].unique())
    print("\nПервые строки для примера:")
    try: from IPython.display import display; display(df_cut.head()) 
    except ImportError: print(df_cut.head())
    print(f"Всего строк: {len(df_cut)}")
else:
    print(f"CSV с вырезанной областью не был создан или пуст.")

## 4. Создание "упрощенного" CSV (`create_csv(is_simple_csv=True)`)
Упрощенный CSV агрегирует детальные ML-метки в более общие категории: `opr_swch`, `abnorm_evnt`, `emerg_evnt`, `normal`.

In [ ]:
output_csv_simple_main_name = "simple_data_main.csv"
print(f"\nЗапуск создания упрощенного CSV (основной выход: {output_csv_simple_main_name}, упрощенный: ..._simple.csv)...")
df_for_simple = converter.create_csv(csv_name=output_csv_simple_main_name, is_cut_out_area=False, is_simple_csv=True)

simple_csv_actual_path = os.path.join(output_dir_conv_csv, os.path.splitext(output_csv_simple_main_name)[0] + "_simple.csv")
if os.path.exists(simple_csv_actual_path):
    print(f"\nСодержимое УПРОЩЕННОГО CSV ({simple_csv_actual_path}):")
    df_simple_loaded = pd.read_csv(simple_csv_actual_path)
    try: from IPython.display import display; display(df_simple_loaded.head()) 
    except ImportError: print(df_simple_loaded.head())
    print(f"\nКолонки упрощенного CSV: {df_simple_loaded.columns.tolist()}")
    
    expected_simple_cols = ['opr_swch', 'abnorm_evnt', 'emerg_evnt', 'normal']
    original_ml_cols = converter.ml_all 
    
    has_simple_cols = all(col in df_simple_loaded.columns for col in expected_simple_cols)
    has_no_original_detailed_ml_cols = not any(col in df_simple_loaded.columns for col in original_ml_cols if col not in ['file_name'])

    print(f"Содержит новые агрегированные колонки: {has_simple_cols}")
    print(f"Удалены старые детальные ML-колонки: {has_no_original_detailed_ml_cols}")

else:
    print(f"Упрощенный CSV ({simple_csv_actual_path}) не был создан.")

## 5. Краткое упоминание `create_csv_for_PDR` и `create_csv_for_SPEF`

Класс `OscillogramToCsvConverter` также содержит методы `create_csv_for_PDR` и `create_csv_for_SPEF`. Они предназначены для создания специализированных CSV-файлов, адаптированных для анализа реле направления мощности (PDR) и однофазных замыканий на землю (SPEF) соответственно. Их использование аналогично `create_csv`, но может требовать специфической конфигурации сигналов в `dict_*.json` файлах или дополнительных входных данных (например, CSV-файла с результатами проверки сигналов для PDR).

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(base_sample_dir_conv):
        shutil.rmtree(base_sample_dir_conv)
        print(f"\nВременная директория {base_sample_dir_conv} удалена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {base_sample_dir_conv}: {e}")